In [1]:
library(anndata)
library(Signac)
library(Seurat)
library(anndata)
library(ggplot2)
library(tidyverse)
library(Signac)
library(Seurat)
#library(EnsDb.Mmusculus.v102)
library(BSgenome.Mmusculus.UCSC.mm10)
library(EnsDb.Mmusculus.v102)
#library(pastecs)
library(stringr)

Attaching SeuratObject

Attaching sp

── Attaching packages ────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.1
✔ purrr   0.3.4     
── Conflicts ───────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()   masks stats::filter()
✖ dplyr::lag()      masks stats::lag()
✖ readr::read_csv() masks anndata::read_csv()
✖ purrr::reduce()   masks Signac::reduce()
Lade nötiges Paket: BSgenome

Lade nötiges Paket: BiocGenerics


Attache Paket: ‘BiocGenerics’


Die folgenden Objekte sind maskiert von ‘package:dplyr’:

    combine, intersect, setdiff, union


Die folgenden Objekte sind maskiert von ‘package:stats’:

    IQR, mad, sd, var, xtabs


Die folgenden Objekte sind maskiert von ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.da

ERROR: Error in library(EnsDb.Mmusculus.v102): es gibt kein Paket namens ‘EnsDb.Mmusculus.v102’


In [2]:
R.version

               _                           
platform       x86_64-apple-darwin13.4.0   
arch           x86_64                      
os             darwin13.4.0                
system         x86_64, darwin13.4.0        
status                                     
major          4                           
minor          1.3                         
year           2022                        
month          03                          
day            10                          
svn rev        81868                       
language       R                           
version.string R version 4.1.3 (2022-03-10)
nickname       One Push-Up                 

In [3]:
devtools::install_github("GreenleafLab/chromVARmotifs")

Skipping install of 'chromVARmotifs' from a github remote, the SHA1 (38bed559) has not changed since last install.
  Use `force = TRUE` to force installation



In [4]:
library("chromVARmotifs")

In [5]:
data("mouse_pwms_v2")

In [6]:
sout <- sapply(strsplit(names(mouse_pwms_v2), split = "_"), function(s) c(s[3]))
mouse_pwms_v2 <- mouse_pwms_v2[match(unique(sout), sout)]

In [8]:
seurat <- readRDS("/Users/dominik.klein/data/seurat_new_annotations.rds")

In [9]:
BSgenome.Mmusculus.UCSC.mm10.renamed <- renameSeqlevels(BSgenome.Mmusculus.UCSC.mm10, value = str_replace(str_replace(seqnames(BSgenome.Mmusculus.UCSC.mm10), pattern = "chr", replacement = ""), pattern = "M", replacement = "MT"))

In [10]:
# add motif information
seurat <- AddMotifs(
  object = seurat,
  genome = BSgenome.Mmusculus.UCSC.mm10.renamed,
  pfm = mouse_pwms_v2
)

Building motif matrix

Finding motif positions

Creating Motif object



In [11]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("chromVAR")

Bioconductor version '3.14' is out-of-date; the current release version '3.16'
  is available with R version '4.2'; see https://bioconductor.org/install

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.18), R 4.1.3 (2022-03-10)

Warning message:
“package(s) not installed when version(s) same as current; use `force = TRUE` to
  re-install: 'chromVAR'”
Old packages: 'BiocManager', 'Hmisc', 'IRkernel', 'MASS', 'Matrix',
  'RcppAnnoy', 'RcppArmadillo', 'RcppEigen', 'Seurat', 'SeuratObject',
  'Signac', 'anndata', 'boot', 'brew', 'broom', 'bslib', 'callr', 'caret',
  'cluster', 'commonmark', 'crul', 'data.table', 'desc', 'devtools', 'digest',
  'dplyr', 'dtplyr', 'e1071', 'evaluate', 'fontawesome', 'forcats', 'foreign',
  'future', 'future.apply', 'gargle', 'gert', 'ggforce', 'ggrepel', 'ggridges',
  'gh', 'gitcreds', 'glmnet', 'glob

In [12]:
library("chromVAR")

In [13]:
seurat <- RunChromVAR(
  object = seurat,
  genome = BSgenome.Mmusculus.UCSC.mm10.renamed
)



Computing GC bias per region

Selecting background regions

Computing deviations from background

Constructing chromVAR assay

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


In [14]:
DefaultAssay(seurat) <- "chromvar"

In [15]:
seurat

An object of class Seurat 
476213 features across 16918 samples within 4 assays 
Active assay: chromvar (884 features, 0 variable features)
 3 other assays present: RNA, ATAC, ATAC_merged
 1 dimensional reduction calculated: lsi

In [16]:
write.csv(seurat[["chromvar"]]@data, "/Users/dominik.klein/data/chromvar_annotations.csv")

In [17]:
sessionInfo()

R version 4.1.3 (2022-03-10)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Big Sur/Monterey 10.16

Matrix products: default
BLAS/LAPACK: /Users/dominik.klein/opt/anaconda3/envs/mm_best_muon_full/lib/libopenblasp-r0.3.20.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] chromVAR_1.16.0                    BiocManager_1.30.18               
 [3] chromVARmotifs_0.2.0               BSgenome.Mmusculus.UCSC.mm10_1.4.3
 [5] BSgenome_1.62.0                    rtracklayer_1.54.0                
 [7] Biostrings_2.62.0                  XVector_0.34.0                    
 [9] GenomicRanges_1.46.1               GenomeInfoDb_1.30.1               
[11] IRanges_2.28.0                     S4Vectors_0.32.4                  
[13] BiocGenerics_0.40.0                forcats_0.5.1                     
[15] stringr_1.4.0                      dplyr_1.0.9          